# CVRP Notebook
This notebook is intended to run the top level code for the CVRP

In [1]:
import networkx as nx

from model import VRP, create_constraints, output_variables
from pricer import VRPPricer
from parse import parse

In [2]:
# Test instance E-n22-k4 provided by parser
G = parse("Instances/E/E-n22-k4.vrp")
# G = parse("Instances/E/E-n23-k3.vrp")
# G = parse("Instances/E/E-n30-k3.vrp")
# G = parse("Instances/E/E-n33-k4.vrp")
# G = parse("Instances/E/E-n51-k5.vrp")
# G = parse("Instances/E/E-n76-k14.vrp")

PARSE: Minimum number of trucks is 4


In [ ]:
model = VRP(G)

# Create pricer
pricer = VRPPricer()
pricer.init_data(G)
pricer.data['elementary'] = True
pricer.data['abort_early'] = True

model.includePricer(pricer, "pricer","does pricing")

create_constraints(model,G)

# model.hideOutput()
model.optimize()
model.hideOutput(quiet=False)
# model.printBestSol()

PRICER_PY: The maximal path length is 32
PRICER_C: Graph data successfully copied to C.
presolving:
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 21 variables (0 bin, 0 int, 0 impl, 21 cont) and 22 constraints
     22 constraints of type <linear>
Presolving Time: 0.00

Starting labelling
goto succesfull
PY PRICING: Found 1660 paths with reduced cost
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  0.4s|     1 |     0 |    54 |     - |  7333k |   0 |1681 |  22 |  22 |   0 |  0 |   0 |   0 |      --      |      --      |    Inf | unknown
Starting labelling
goto succesfull
PY PRICING: Found 38737 paths with reduced cost


In [4]:
output_variables(model, pricer)

The solution contains the following paths: 
0.125 * x334: (0, 7, 5, 7, 5, 0)
0.25 * x4238: (0, 16, 14, 16, 14, 0)
0.5 * x5108: (0, 13, 11, 13, 16, 0)
0.22222222222222215 * x13553: (0, 10, 8, 3, 4, 3, 4, 3, 8, 0)
0.33333333333333337 * x13572: (0, 19, 21, 19, 14, 0)
0.25 * x13589: (0, 9, 5, 7, 5, 9, 0)
0.16666666666666666 * x13609: (0, 14, 20, 21, 19, 21, 0)
0.2500000000000001 * x13656: (0, 11, 4, 3, 4, 11, 0)
0.25 * x13659: (0, 9, 7, 5, 7, 9, 12, 0)
0.16666666666666663 * x13779: (0, 17, 21, 19, 21, 17, 0)
0.6666666666666667 * x13813: (0, 17, 20, 18, 15, 12, 0)
0.1333333333333333 * x14327: (0, 15, 18, 20, 18, 15, 0)
0.19444444444444442 * x14790: (0, 10, 6, 2, 1, 2, 1, 6, 10, 0)
0.16666666666666669 * x14791: (0, 10, 8, 6, 2, 1, 2, 1, 6, 8, 10, 0)
